# SMA+EMA+BB+ATR Strategy

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz


# PARAMETERS
symbol = "ITC.NS"
capital = 100000         # ₹1,00,000 starting capital
leverage = 5
atr_period = 14
atr_mult = 1.5            # ATR multiplier for stop loss
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')


# Download 5-minute data (~60 days)
data = yf.download(symbol, interval='5m', period='60d', progress=False, prepost=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()


# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_upper, bb_middle, bb_lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'] = bb_upper
data['BB_LOWER'] = bb_lower
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)

# Helper functions for brokerage/costs
def brokerage(order_value):
    raw_brokerage = min(20, order_value * 0.001)
    if raw_brokerage < 5:
        return min(5, order_value * 0.025)
    return raw_brokerage


def stt(trade_val): return trade_val * 0.00025   # 0.025% on sell side only
def stamp_duty(trade_val): return trade_val * 0.00003   # 0.003% on buy side only
def exchange_charges(trade_val): return trade_val * 0.0000297   # 0.00297% both sides (NSE rate)
def sebi_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def ipf_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def gst(broker, exch, sebi): return 0.18 * (broker + exch + sebi)


# Trade simulation state
trades = []
position = None
stop_loss = None
max_price = None
min_price = None


for i in range(1, len(data)):
    row = data.iloc[i]
    prev = data.iloc[i-1]
    ts = row.name


    # Force exit before 15:00 IST
    if ts.time() >= force_exit_time:
        if position:
            exit_price = row['Close']
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            # PnL calculation
            if position['direction'] == 'long':
                gross_pnl = (exit_price - position['entry_price']) * qty
                stt_ = 0                            # No STT on buy
                stamp = stamp_duty(entry_val)      # Stamp duty on buy side (entry)
            else:  # short
                gross_pnl = (position['entry_price'] - exit_price) * qty
                stt_ = stt(entry_val)              # STT on sell side (entry)
                stamp = stamp_duty(exit_val)       # Stamp duty on buy side (exit)

            # Charges
            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)
            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': position['direction'],
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,
            
                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None
        continue


    # Only consider bars with valid ATR and indicators
    if pd.isna(row['ATR']) or pd.isna(row['SMA5']) or pd.isna(row['EMA9']) or pd.isna(row['BB_UPPER']) or pd.isna(row['BB_LOWER']):
        continue


    # Entry signals
    long_signal = (prev['Close'] < prev['SMA5'] and prev['Close'] < prev['EMA9'] and
                   row['Close'] > row['SMA5'] and row['Close'] > row['EMA9'] and
                   row['Close'] > row['BB_LOWER'])
    short_signal = (prev['Close'] > prev['SMA5'] and prev['Close'] > prev['EMA9'] and
                    row['Close'] < row['SMA5'] and row['Close'] < row['EMA9'] and
                    row['Close'] < row['BB_UPPER'])


    price = row['Close']


    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty == 0:
            continue
        if long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'long'}
            stop_loss = price - atr_mult * row['ATR']  # ATR-based stop loss
            max_price = price
        elif short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'short'}
            stop_loss = price + atr_mult * row['ATR']
            min_price = price


    elif position['direction'] == 'long':
        max_price = max(max_price, price)
        # Trail stop loss: highest price - ATR * multiplier
        new_stop = max_price - atr_mult * row['ATR']
        stop_loss = max(stop_loss, new_stop)
        if price <= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            gross_pnl = (exit_price - position['entry_price']) * qty
            stt_ = 0
            stamp = stamp_duty(entry_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'long',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


    elif position['direction'] == 'short':
        min_price = min(min_price, price)
        # Trail stop loss: lowest price + ATR * multiplier
        new_stop = min_price + atr_mult * row['ATR']
        stop_loss = min(stop_loss, new_stop)
        if price >= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = price * qty

            gross_pnl = (position['entry_price'] - exit_price) * qty
            stt_ = stt(entry_val)
            stamp = stamp_duty(exit_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'short',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


# Export trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('intraday_scalping_trades_atr_stop.csv', index=False)


# Performance metrics
net_pnl = trades_df['net_pnl'].sum()
win_trades = (trades_df['net_pnl'] > 0).sum()
total_trades = len(trades_df)
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0
avg_trade_pnl = trades_df['net_pnl'].mean() if total_trades > 0 else 0
cum_pnl = trades_df['net_pnl'].cumsum()
max_drawdown = (cum_pnl.cummax() - cum_pnl).max() if total_trades > 0 else 0
total_costs = trades_df['costs'].sum()


print("Final Performance Metrics:")
print(f"Net PnL: ₹{net_pnl:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_trade_pnl:,.2f}")
print(f"Max drawdown: ₹{max_drawdown:,.2f}")
print(f"Number of trades: {total_trades}")
print(f"Total brokerage/charges paid: ₹{total_costs:,.2f}")
print("Trade details exported to 'intraday_scalping_trades_atr_stop.csv'")


Final Performance Metrics:
Net PnL: ₹-22,355.22
Win rate: 38.46%
Avg trade PnL: ₹-122.83
Max drawdown: ₹29,389.50
Number of trades: 182
Total brokerage/charges paid: ₹28,212.20
Trade details exported to 'intraday_scalping_trades_atr_stop.csv'


# SMA+EMA+BB+ATR+RSI+MACD

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz


# PARAMETERS
symbol = "ITC.NS"
capital = 100000           # ₹1,00,000 starting capital
leverage = 5
atr_period = 14
atr_mult = 1.5            # ATR multiplier for stop loss
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')


# Download 5-minute data (~60 days)
data = yf.download(symbol, interval='5m', period='60d', progress=False, prepost=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()


# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_upper, bb_middle, bb_lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'] = bb_upper
data['BB_LOWER'] = bb_lower
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)

# Additional indicators: RSI and MACD
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
macd, macdsignal, macdhist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'] = macd
data['MACD_signal'] = macdsignal


# Helper functions for brokerage/costs
def brokerage(order_value):
    raw_brokerage = min(20, order_value * 0.001)
    if raw_brokerage < 5:
        return min(5, order_value * 0.025)
    return raw_brokerage


def stt(trade_val): return trade_val * 0.00025   # 0.025% on sell side only
def stamp_duty(trade_val): return trade_val * 0.00003   # 0.003% on buy side only
def exchange_charges(trade_val): return trade_val * 0.0000297   # 0.00297% both sides (NSE rate)
def sebi_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def ipf_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def gst(broker, exch, sebi): return 0.18 * (broker + exch + sebi)


# Trade simulation state
trades = []
position = None
stop_loss = None
max_price = None
min_price = None


for i in range(1, len(data)):
    row = data.iloc[i]
    prev = data.iloc[i-1]
    ts = row.name

    # Force exit before 15:00 IST
    if ts.time() >= force_exit_time:
        if position:
            exit_price = row['Close']
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            # PnL calculation
            if position['direction'] == 'long':
                gross_pnl = (exit_price - position['entry_price']) * qty
                stt_ = 0                            # No STT on buy
                stamp = stamp_duty(entry_val)      # Stamp duty on buy side (entry)
            else:  # short
                gross_pnl = (position['entry_price'] - exit_price) * qty
                stt_ = stt(entry_val)              # STT on sell side (entry)
                stamp = stamp_duty(exit_val)       # Stamp duty on buy side (exit)

            # Charges
            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)
            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': position['direction'],
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None
        continue


    # Skip bars without full indicator data
    if pd.isna(row['ATR']) or pd.isna(row['SMA5']) or pd.isna(row['EMA9']) or pd.isna(row['BB_UPPER']) or pd.isna(row['BB_LOWER']) or pd.isna(row['RSI']) or pd.isna(row['MACD']) or pd.isna(row['MACD_signal']):
        continue


    # Entry signals with additional RSI and MACD filters
    long_signal = (
        prev['Close'] < prev['SMA5']
        and prev['Close'] < prev['EMA9']
        and row['Close'] > row['SMA5']
        and row['Close'] > row['EMA9']
        and row['Close'] > row['BB_LOWER']
        and 40 < row['RSI'] < 80
        and row['MACD'] > row['MACD_signal']
    )
    short_signal = (
        prev['Close'] > prev['SMA5']
        and prev['Close'] > prev['EMA9']
        and row['Close'] < row['SMA5']
        and row['Close'] < row['EMA9']
        and row['Close'] < row['BB_UPPER']
        and 20 < row['RSI'] < 60
        and row['MACD'] < row['MACD_signal']
    )


    price = row['Close']


    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty == 0:
            continue
        if long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'long'}
            stop_loss = price - atr_mult * row['ATR']  # ATR-based stop loss
            max_price = price
        elif short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'short'}
            stop_loss = price + atr_mult * row['ATR']
            min_price = price


    elif position['direction'] == 'long':
        max_price = max(max_price, price)
        # Trail stop loss: highest price - ATR * multiplier
        new_stop = max_price - atr_mult * row['ATR']
        stop_loss = max(stop_loss, new_stop)
        if price <= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            gross_pnl = (exit_price - position['entry_price']) * qty
            stt_ = 0
            stamp = stamp_duty(entry_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'long',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None


    elif position['direction'] == 'short':
        min_price = min(min_price, price)
        # Trail stop loss: lowest price + ATR * multiplier
        new_stop = min_price + atr_mult * row['ATR']
        stop_loss = min(stop_loss, new_stop)
        if price >= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = price * qty

            gross_pnl = (position['entry_price'] - exit_price) * qty
            stt_ = stt(entry_val)
            stamp = stamp_duty(exit_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'short',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None


# Export trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('intraday_scalping_trades_atr_stop_with_filters.csv', index=False)


# Performance metrics
net_pnl = trades_df['net_pnl'].sum()
win_trades = (trades_df['net_pnl'] > 0).sum()
total_trades = len(trades_df)
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0
avg_trade_pnl = trades_df['net_pnl'].mean() if total_trades > 0 else 0
cum_pnl = trades_df['net_pnl'].cumsum()
max_drawdown = (cum_pnl.cummax() - cum_pnl).max() if total_trades > 0 else 0
total_costs = trades_df['total_cost'].sum()


print("Final Performance Metrics:")
print(f"Net PnL: ₹{net_pnl:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_trade_pnl:,.2f}")
print(f"Max drawdown: ₹{max_drawdown:,.2f}")
print(f"Number of trades: {total_trades}")
print(f"Total brokerage/charges paid: ₹{total_costs:,.2f}")
print("Trade details exported to 'intraday_scalping_trades_atr_stop_with_filters.csv'")


Final Performance Metrics:
Net PnL: ₹-24,081.95
Win rate: 36.36%
Avg trade PnL: ₹-182.44
Max drawdown: ₹35,943.03
Number of trades: 132
Total brokerage/charges paid: ₹19,493.87
Trade details exported to 'intraday_scalping_trades_atr_stop_with_filters.csv'


# SMA+EMA+BB+ATR+RSI+MACD+VWAP

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "ITC.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone("Asia/Kolkata")

# Thresholds
rsi_low, rsi_high = 40, 80
mfi_low, mfi_high = None, None  # not used

# COST MODEL
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb
def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# FETCH DATA
df = yf.download(symbol, interval="5m", period="60d", progress=False, auto_adjust=True, multi_level_index=False)
df = df.tz_convert("Asia/Kolkata")
df = df.between_time(intraday_start, intraday_end).dropna()

# INDICATORS
df["SMA5"] = talib.SMA(df["Close"], 5)
df["EMA9"] = talib.EMA(df["Close"], 9)
bb_u, _, bb_l = talib.BBANDS(df["Close"], 20, 2, 2)
df["BB_UPPER"], df["BB_LOWER"] = bb_u, bb_l
df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"], atr_period)
df["RSI"] = talib.RSI(df["Close"], 14)
macd, macd_sig, _ = talib.MACD(df["Close"], 12, 26, 9)
df["MACD"], df["MACD_signal"] = macd, macd_sig

# VWAP
typ = (df["High"] + df["Low"] + df["Close"]) / 3
cum_vol = df["Volume"].cumsum()
cum_pv  = (typ * df["Volume"]).cumsum()
df["VWAP"] = cum_pv / cum_vol

# SIMULATE TRADES
trades = []
pos = None

for i in range(1, len(df)):
    row, prev = df.iloc[i], df.iloc[i-1]
    t = row.name.time()

    # Force exit at/after 15:00
    if pos and t >= force_exit_time:
        px, tx = row["Close"], row.name
        q = pos["qty"]
        ev, xv = pos["entry_price"]*q, px*q
        gross = (px-pos["entry_price"])*q if pos["dir"]=="long" else (pos["entry_price"]-px)*q
        sttax = 0 if pos["dir"]=="long" else stt(ev)
        stduty= stamp(ev) if pos["dir"]=="long" else stamp(xv)
        brk = brokerage(ev)+brokerage(xv)
        exc = exch(ev)+exch(xv)
        sb  = sebi(ev)+sebi(xv)
        ip  = ipf(ev)+ipf(xv)
        gstx= gst(brk, exc, sb)
        cost= brk+exc+sb+ip+sttax+stduty+gstx
        net = gross - cost
        trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                       "dir":pos["dir"],"entry":pos["entry_price"],
                       "exit":px,"qty":q,"gross":gross,"net":net,
                       "brokerage":brk,"stt":sttax,"stamp":stduty,
                       "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
        pos = None

    # Skip if any required indicator is NaN
    req = ["SMA5","EMA9","BB_UPPER","BB_LOWER","ATR","RSI","MACD","MACD_signal","VWAP"]
    if row[req].isna().any(): continue

    # Entry signals
    long_sig = (
        prev["Close"]<prev["SMA5"] and prev["Close"]<prev["EMA9"] and
        row["Close"]>row["SMA5"] and row["Close"]>row["EMA9"] and
        row["Close"]>row["BB_LOWER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]>row["MACD_signal"] and
        row["Close"]>row["VWAP"]
    )
    short_sig = (
        prev["Close"]>prev["SMA5"] and prev["Close"]>prev["EMA9"] and
        row["Close"]<row["SMA5"] and row["Close"]<row["EMA9"] and
        row["Close"]<row["BB_UPPER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]<row["MACD_signal"] and
        row["Close"]<row["VWAP"]
    )

    price = row["Close"]
    if not pos:
        maxv = capital * leverage
        q = int(maxv//price)
        if q and long_sig:
            pos = {"entry_time":row.name,"entry_price":price,"qty":q,"dir":"long"}
            sl = price - atr_mult*row["ATR"]; maxp = price
        elif q and short_sig:
            pos = {"entry_time":row.name,"entry_price":price,"qty":q,"dir":"short"}
            sl = price + atr_mult*row["ATR"]; minp = price
    else:
        # Trailing stop
        if pos["dir"]=="long":
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult*row["ATR"])
            if price <= sl:
                px, tx = price, row.name
                q=pos["qty"]; ev=pos["entry_price"]*q; xv=px*q
                gross=(px-pos["entry_price"])*q
                sttax, stduty = 0, stamp(ev)
                brk=brokerage(ev)+brokerage(xv)
                exc=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,exc,sb)
                cost=brk+exc+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                               "dir":"long","entry":pos["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
                pos=None
        else:
            minp=min(minp, price)
            sl=min(sl, minp + atr_mult*row["ATR"])
            if price >= sl:
                px, tx=price, row.name
                q=pos["qty"]; ev=pos["entry_price"]*q; xv=px*q
                gross=(pos["entry_price"]-px)*q
                sttax, stduty=stt(ev), stamp(xv)
                brk=brokerage(ev)+brokerage(xv)
                exc=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,exc,sb)
                cost=brk+exc+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                               "dir":"short","entry":pos["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
                pos=None

# OUTPUT
out = pd.DataFrame(trades)
out.to_csv("scalp_SMA_EMA_BB_ATR_RSI_MACD_VWAP.csv", index=False)

net = out["net"].sum()
wins = (out["net"]>0).sum(); n=len(out)
winr = wins/n*100 if n else 0
avgp= out["net"].mean() if n else 0
cum= out["net"].cumsum(); mdd=(cum.cummax()-cum).max() if n else 0
costs= out[["brokerage","stt","stamp","exchange","sebi","ipf","gst"]].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {winr:.2f}%")
print(f"Avg trade PnL: ₹{avgp:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {n}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")


Net PnL: ₹-8,909.30
Win rate: 40.58%
Avg trade PnL: ₹-129.12
Max drawdown: ₹19,139.47
Number of trades: 69
Total brokerage/charges paid: ₹13,606.17


# SMA+EMA+BB+ATR+RSI+MACD+Stochastic

In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "ITC.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone("Asia/Kolkata")

# RSI & Stochastic thresholds
rsi_low, rsi_high = 40, 80
mfi_low, mfi_high = None, None  # Not used
sto_k, sto_d = 5, 3

# COST MODEL
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb
def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# FETCH AND PREPARE DATA
df = yf.download(symbol, interval="5m", period="60d", progress=False, auto_adjust=True, multi_level_index=False)
df = df.tz_convert("Asia/Kolkata")
df = df.between_time(intraday_start, intraday_end).dropna()

# INDICATORS
df["SMA5"] = talib.SMA(df["Close"], 5)
df["EMA9"] = talib.EMA(df["Close"], 9)
bb_u, _, bb_l = talib.BBANDS(df["Close"], 20, 2, 2)
df["BB_UPPER"], df["BB_LOWER"] = bb_u, bb_l
df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"], atr_period)
df["RSI"] = talib.RSI(df["Close"], 14)
macd, macd_sig, _ = talib.MACD(df["Close"], 12, 26, 9)
df["MACD"], df["MACD_signal"] = macd, macd_sig
sto_k_vals, sto_d_vals = talib.STOCH(
    df["High"], df["Low"], df["Close"],
    fastk_period=sto_k, slowk_period=sto_d,
    slowd_period=sto_d
)
df["STOCHK"], df["STOCHD"] = sto_k_vals, sto_d_vals

# TRADE SIMULATION
trades = []
position = None

for i in range(1, len(df)):
    row, prev = df.iloc[i], df.iloc[i-1]
    t = row.name.time()

    # Forced exit at or after 15:00 IST
    if position and t >= force_exit_time:
        px, tx = row["Close"], row.name
        q = position["qty"]
        ev, xv = position["entry_price"]*q, px*q
        gross = (px - position["entry_price"])*q if position["dir"]=="long" else (position["entry_price"] - px)*q
        sttax = 0 if position["dir"]=="long" else stt(ev)
        stduty = stamp(ev) if position["dir"]=="long" else stamp(xv)
        brk = brokerage(ev)+brokerage(xv)
        excv = exch(ev)+exch(xv)
        sb = sebi(ev)+sebi(xv)
        ip = ipf(ev)+ipf(xv)
        gstx = gst(brk, excv, sb)
        cost = brk+excv+sb+ip+sttax+stduty+gstx
        net = gross - cost
        trades.append({
            "entry_time": position["entry_time"], "exit_time": tx,
            "dir": position["dir"], "entry": position["entry_price"],
            "exit": px, "qty": q, "gross": gross, "net": net,
            "brokerage": brk, "stt": sttax, "stamp": stduty,
            "exchange": excv, "sebi": sb, "ipf": ip, "gst": gstx
        })
        position = None

    # Skip if any indicator is NaN
    if row[["SMA5","EMA9","BB_UPPER","BB_LOWER","ATR","RSI","MACD","MACD_signal","STOCHK","STOCHD"]].isna().any():
        continue

    # ENTRY SIGNALS
    long_sig = (
        prev["Close"]<prev["SMA5"] and prev["Close"]<prev["EMA9"] and
        row["Close"]>row["SMA5"] and row["Close"]>row["EMA9"] and
        row["Close"]>row["BB_LOWER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]>row["MACD_signal"] and
        row["STOCHK"]<20 and row["STOCHK"]>row["STOCHD"]
    )
    short_sig = (
        prev["Close"]>prev["SMA5"] and prev["Close"]>prev["EMA9"] and
        row["Close"]<row["SMA5"] and row["Close"]<row["EMA9"] and
        row["Close"]<row["BB_UPPER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]<row["MACD_signal"] and
        row["STOCHK"]>80 and row["STOCHK"]<row["STOCHD"]
    )

    price = row["Close"]
    if not position:
        max_val = capital * leverage
        qty = int(max_val // price)
        if qty and long_sig:
            position = {"entry_time": row.name, "entry_price": price, "qty": qty, "dir": "long"}
            sl = price - atr_mult * row["ATR"]
            maxp = price
        elif qty and short_sig:
            position = {"entry_time": row.name, "entry_price": price, "qty": qty, "dir": "short"}
            sl = price + atr_mult * row["ATR"]
            minp = price
    else:
        # TRAILING STOP
        if position["dir"]=="long":
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult * row["ATR"])
            if price <= sl:
                px, tx = price, row.name
                q=position["qty"]; ev=position["entry_price"]*q; xv=px*q
                gross=(px-position["entry_price"])*q
                sttax, stduty=0, stamp(ev)
                brk=brokerage(ev)+brokerage(xv)
                excv=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,excv,sb)
                cost=brk+excv+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":position["entry_time"],"exit_time":tx,
                               "dir":"long","entry":position["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":excv,"sebi":sb,"ipf":ip,"gst":gstx})
                position=None
        else:
            minp=min(minp, price)
            sl=min(sl, minp + atr_mult*row["ATR"])
            if price >= sl:
                px, tx=price, row.name
                q=position["qty"]; ev=position["entry_price"]*q; xv=px*q
                gross=(position["entry_price"]-px)*q
                sttax, stduty=stt(ev), stamp(xv)
                brk=brokerage(ev)+brokerage(xv)
                excv=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,excv,sb)
                cost=brk+excv+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":position["entry_time"],"exit_time":tx,
                               "dir":"short","entry":position["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":excv,"sebi":sb,"ipf":ip,"gst":gstx})
                position=None

# EXPORT & METRICS
out = pd.DataFrame(trades)
out.to_csv("scalp_SMA_EMA_BB_ATR_RSI_MACD_STOCH.csv", index=False)

net = out["net"].sum()
wins = (out["net"]>0).sum(); n = len(out)
win_rate = wins/n*100 if n else 0
avg_pnl = out["net"].mean() if n else 0
cum = out["net"].cumsum(); mdd = (cum.cummax()-cum).max() if n else 0
costs = out[["brokerage","stt","stamp","exchange","sebi","ipf","gst"]].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_pnl:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {n}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")


Net PnL: ₹-1,582.59
Win rate: 0.00%
Avg trade PnL: ₹-791.30
Max drawdown: ₹461.17
Number of trades: 2
Total brokerage/charges paid: ₹323.81


# SMA+EMA+BB+ATR+RSI+MACD+MFI

In [6]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "ITC.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')

# Indicator thresholds
rsi_long_low, rsi_long_high = 40, 80
rsi_short_low, rsi_short_high = 20, 60
mfi_low, mfi_high = 20, 80  # MFI oversold/overbought

# Download data
data = yf.download(symbol, interval='5m', period='60d',
                   progress=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()

# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_u, bb_m, bb_l = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'], data['BB_LOWER'] = bb_u, bb_l
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
macd, macd_sig, _ = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'], data['MACD_signal'] = macd, macd_sig
data['MFI'] = talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=14)

# Cost model functions
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb

def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# Simulate trades
trades = []
position = None

for i in range(1, len(data)):
    row, prev = data.iloc[i], data.iloc[i-1]
    ts = row.name

    # Forced exit at or after 15:00
    if ts.time() >= force_exit_time and position:
        exit_price, exit_time = row['Close'], ts
        qty = position['qty']
        ent_val = position['entry_price'] * qty
        ext_val = exit_price * qty
        if position['dir']=='long':
            gross = (exit_price - position['entry_price']) * qty
            sttax, stduty = 0, stamp(ent_val)
        else:
            gross = (position['entry_price'] - exit_price) * qty
            sttax, stduty = stt(ent_val), stamp(ext_val)
        brk = brokerage(ent_val) + brokerage(ext_val)
        exc = exch(ent_val) + exch(ext_val)
        sb  = sebi(ent_val) + sebi(ext_val)
        ip  = ipf(ent_val) + ipf(ext_val)
        gstx= gst(brk, exc, sb)
        total_cost = brk + exc + sb + ip + sttax + stduty + gstx
        net = gross - total_cost
        trades.append({
            'entry_time': position['entry_time'], 'exit_time': exit_time,
            'dir': position['dir'], 'entry': position['entry_price'],
            'exit': exit_price, 'qty': qty, 'gross': gross, 'net': net,
            'brokerage': brk, 'stt': sttax, 'stamp': stduty,
            'exchange': exc, 'sebi': sb, 'ipf': ip, 'gst': gstx
        })
        position = None
        continue

    # Skip if any indicator is NaN
    if row[['SMA5','EMA9','BB_UPPER','BB_LOWER','ATR','RSI','MACD','MACD_signal','MFI']].isna().any():
        continue

    # Entry conditions
    long_signal = (
        prev['Close'] < prev['SMA5'] and prev['Close'] < prev['EMA9'] and
        row['Close'] > row['SMA5'] and row['Close'] > row['EMA9'] and
        row['Close'] > row['BB_LOWER'] and
        rsi_long_low < row['RSI'] < rsi_long_high and
        row['MACD'] > row['MACD_signal'] and
        row['MFI'] < mfi_low
    )
    short_signal = (
        prev['Close'] > prev['SMA5'] and prev['Close'] > prev['EMA9'] and
        row['Close'] < row['SMA5'] and row['Close'] < row['EMA9'] and
        row['Close'] < row['BB_UPPER'] and
        rsi_short_low < row['RSI'] < rsi_short_high and
        row['MACD'] < row['MACD_signal'] and
        row['MFI'] > mfi_high
    )

    price = row['Close']
    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty and long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'dir': 'long'}
            sl = price - atr_mult * row['ATR']
            maxp = price
        elif qty and short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'dir': 'short'}
            sl = price + atr_mult * row['ATR']
            minp = price
    else:
        # Trailing stop logic
        if position['dir']=='long':
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult * row['ATR'])
            if price <= sl:
                exit_price, exit_time = price, ts
                qty = position['qty']
                ent_val, ext_val = position['entry_price']*qty, exit_price*qty
                gross = (exit_price - position['entry_price']) * qty
                sttax, stduty = 0, stamp(ent_val)
                brk = brokerage(ent_val) + brokerage(ext_val)
                exc = exch(ent_val) + exch(ext_val)
                sb  = sebi(ent_val) + sebi(ext_val)
                ip  = ipf(ent_val) + ipf(ext_val)
                gstx= gst(brk, exc, sb)
                total_cost = brk + exc + sb + ip + sttax + stduty + gstx
                net = gross - total_cost
                trades.append({
                    'entry_time': position['entry_time'], 'exit_time': exit_time,
                    'dir':'long','entry':position['entry_price'],'exit':exit_price,
                    'qty':qty,'gross':gross,'net':net,'brokerage':brk,
                    'stt':sttax,'stamp':stduty,'exchange':exc,
                    'sebi':sb,'ipf':ip,'gst':gstx
                })
                position = None
        else:
            minp = min(minp, price)
            sl = min(sl, minp + atr_mult * row['ATR'])
            if price >= sl:
                exit_price, exit_time = price, ts
                qty = position['qty']
                ent_val, ext_val = position['entry_price']*qty, exit_price*qty
                gross = (position['entry_price'] - exit_price) * qty
                sttax, stduty = stt(ent_val), stamp(ext_val)
                brk = brokerage(ent_val) + brokerage(ext_val)
                exc = exch(ent_val) + exch(ext_val)
                sb  = sebi(ent_val) + sebi(ext_val)
                ip  = ipf(ent_val) + ipf(ext_val)
                gstx= gst(brk, exc, sb)
                total_cost = brk + exc + sb + ip + sttax + stduty + gstx
                net = gross - total_cost
                trades.append({
                    'entry_time': position['entry_time'], 'exit_time': exit_time,
                    'dir':'short','entry':position['entry_price'],'exit':exit_price,
                    'qty':qty,'gross':gross,'net':net,'brokerage':brk,
                    'stt':sttax,'stamp':stduty,'exchange':exc,
                    'sebi':sb,'ipf':ip,'gst':gstx
                })
                position = None

# Export and performance
df = pd.DataFrame(trades)
df.to_csv('scalping_sma_ema_bb_atr_rsi_macd_mfi.csv', index=False)

net = df['net'].sum()
wins = (df['net']>0).sum(); total = len(df)
winr = wins/total*100 if total else 0
avg = df['net'].mean() if total else 0
cum = df['net'].cumsum(); mdd = (cum.cummax()-cum).max() if total else 0
costs = df[['brokerage','stt','stamp','exchange','sebi','ipf','gst']].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {winr:.2f}%")
print(f"Avg trade PnL: ₹{avg:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {total}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")
print("Detailed trades in 'scalping_sma_ema_bb_atr_rsi_macd_mfi.csv'")


Net PnL: ₹-927.86
Win rate: 33.33%
Avg trade PnL: ₹-309.29
Max drawdown: ₹1,356.87
Number of trades: 3
Total brokerage/charges paid: ₹673.19
Detailed trades in 'scalping_sma_ema_bb_atr_rsi_macd_mfi.csv'


# SMA+EMA+BB+ATR+RSI+MACD+ADX

It downloads 5-minute candles for any NSE ticker from Yahoo-Finance with yfinance.

Signals are generated from RSI-14, EMA-9/21, Bollinger-Bands-20-2 and MACD-12-26-9.

Realistic slippage (≈0.02%) and Groww intraday costs (₹20 or 0.1% per order + statutory charges) are debited on every entry/exit.

Risk is capped at 1% of capital per trade, 1.5% stop-loss, 2.5% target, max-3 consecutive losses or 1% daily draw-down.

No external back-test framework is used—everything is pure-python/pandas/TA-Lib.

At the end you get a CSV of every trade plus a performance summary.

In [ ]:
"""
INDIAN NSE 5-MIN SCALPING ALGO
Author : you
Broker : Groww
Capital: ₹1,00,000
"""

# ──────────────────────── 1. IMPORTS ────────────────────────────────
import yfinance as yf
import talib, pandas as pd, numpy as np
from datetime import datetime, timedelta

# ────────────────── 2. GROWW BROKERAGE + SLIPPAGE ───────────────────
def groww_cost(value, intraday=True):
    pct = value*0.001                 # 0.1 %
    brk = max(5, min(20, pct))        # ₹5 min, ₹20 max
    stt  = value*0.00025 if intraday else value*0.001
    stamp= value*0.00003 if intraday else value*0.00015
    exch = value*0.0000297
    sebi = value*0.000001
    gst  = 0.18*(brk+exch+sebi)
    return brk+stt+stamp+exch+sebi+gst

def slip(price, vol, buy=True):
    s = max(.10, min(2.0, price*0.0002*max(0.5,2e5/max(vol,1))))
    return price+s if buy else price-s

# ───────────────────────── 3. STRATEGY  ─────────────────────────────
class Scalper:
    def __init__(self, cash=1_00_000, risk=.01, dayloss=.01):
        self.cash0 = cash; self.cash=cash
        self.risk=risk; self.dayloss=dayloss
        self.trades, self.daypnl, self.lossrun = [],{},0

    # ---------- indicators ----------
    def ind(self, df):
        c,h,l,v = df.Close.values, df.High.values, df.Low.values, df.Volume.values
        out={}
        out['rsi']=talib.RSI(c,14)
        out['macd'],out['macds'],_=talib.MACD(c,12,26,9)
        out['bbU'],out['bbM'],out['bbL']=talib.BBANDS(c,20,2,2)
        out['ema9']=talib.EMA(c,9); out['ema21']=talib.EMA(c,21)
        out['vwap']=np.cumsum(((h+l+c)/3)*v)/np.cumsum(v)
        return out

    # ---------- signals ----------
    def sig(self, i, df, ind):
        c,r,ma,mb,ml = df.Close[i],ind['rsi'][i],ind['ema9'][i],ind['ema21'][i],ind['bbM'][i]
        mac,macs = ind['macd'][i],ind['macds'][i]
        if np.isnan([r,ma,mb,mac]).any(): return 0
        bulls = sum([r<40 and r>ind['rsi'][i-1],
                     mac>macs, c<ml, ma>mb])
        bears = sum([r>60 and r<ind['rsi'][i-1],
                     mac<macs, c>ml, ma<mb ])
        return 1 if bulls>=2 else -1 if bears>=2 else 0

    # ---------- back-test single symbol ----------
    def backtest(self,tkr,days=7):
        df=yf.download(tkr, start=datetime.now()-timedelta(days), interval='5m', progress=False, auto_adjust=True)
        if df.empty: return
        if isinstance(df.columns,pd.MultiIndex):
            df.columns=[c[0] for c in df.columns]
        df=df.dropna(); ind=self.ind(df); pos=None
        for i in range(50,len(df)):
            day=str(df.index[i].date())
            if self.daypnl.get(day,-0)>-self.cash0*self.dayloss or self.lossrun<3:
                s=self.sig(i,df,ind); price=df.Close[i]; vol=df.Volume[i]
                # exit?
                if pos:
                    pnl=(price-pos['price'])*pos['qty']*(-1 if pos['short'] else 1)
                    stop= -self.cash*self.risk if pos['short'] else self.cash*self.risk
                    tgt=  self.cash*self.risk*2.5
                    if (pnl<=stop or pnl>=tgt or s==-pos['dir']):
                        exitp=slip(price,vol,pos['short'])
                        gross=(exitp-pos['price'])*pos['qty']*(-1 if pos['short'] else 1)
                        net=gross-groww_cost(pos['price']*pos['qty'])-groww_cost(exitp*pos['qty'])
                        self.cash+=net; self.daypnl[day]=self.daypnl.get(day,0)+net
                        self.lossrun= self.lossrun+1 if net<0 else 0
                        self.trades.append([tkr,pos['price'],exitp,pos['qty'],net,pos['short']])
                        pos=None
                # entry?
                if not pos and s:
                    sl= self.cash*self.risk; qty=int(sl/(price*.015))
                    qty=min(qty,int(self.cash*.8/price))
                    if qty>0:
                        ent=slip(price,vol,s==-1); cost=groww_cost(ent*qty)
                        if ent*qty+cost<=self.cash:
                            self.cash-=cost; self.lossrun=0
                            pos={'price':ent,'qty':qty,'short':s==-1,'dir':s}
        return

# ──────────────────── 4. RUN & EXPORT RESULTS ───────────────────────
stk=['ITC.NS']          # add your list
bot=Scalper()
for s in stk: bot.backtest(s,7)

# results ------------------------------------------------------------
if bot.trades:
    cols=['Symbol','Entry','Exit','Qty','Net₹','Short?']
    trades=pd.DataFrame(bot.trades,columns=cols)
    print(trades)
    print(f"\nStart ₹{bot.cash0:,.0f}  →  End ₹{bot.cash:,.0f}  |  P&L ₹{bot.cash-bot.cash0:,.0f}")
    # print(f"Win rate {(trades.Net₹>0).mean()*100:.1f}%  |  Avg ₹{trades.Net₹.mean():.0f}")
    print(f"Win rate {(trades['Net₹']>0).mean()*100:.1f}%  |  Avg ₹{trades['Net₹'].mean():.0f}")
    trades.to_csv('nse_scalping_trades.csv',index=False)
else:
    print("No trades triggered – relax entry rules or widen look-back.")



     Symbol       Entry        Exit  Qty        Net₹  Short?
0    ITC.NS  416.599760  416.555304  159  -96.159377   False
1    ITC.NS  416.555304  416.450554  159 -105.738699   False
2    ITC.NS  416.450554  416.549263  159  -73.388602   False
3    ITC.NS  416.549263  416.593309  159  -82.086910   False
4    ITC.NS  416.593309  416.440891  159 -113.319439   False
..      ...         ...         ...  ...         ...     ...
139  ITC.NS  412.170297  412.300012  132  -98.737283    True
140  ITC.NS  412.100012  412.250006  132  -61.810686   False
141  ITC.NS  412.450006  412.377344  132  -72.038471    True
142  ITC.NS  412.022680  411.549994  132 -143.971989   False
143  ITC.NS  411.749994  413.450012  131 -304.086809    True

[144 rows x 6 columns]

Start ₹100,000  →  End ₹81,067  |  P&L ₹-18,933
Win rate 5.6%  |  Avg ₹-89


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_59692/4123455877.py:43: RuntimeWarning: invalid value encountered in divide
  out['vwap']=np.cumsum(((h+l+c)/3)*v)/np.cumsum(v)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_59692/4123455877.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c,r,ma,mb,ml = df.Close[i],ind['rsi'][i],ind['ema9'][i],ind['ema21'][i],ind['bbM'][i]
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_59692/4123455877.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  s=self.sig(i,df,ind); price=df.Close[i]; vol=df.Volume[i]


In [ ]:
"""
INDIAN NSE 5-Min Scalping Algorithm with 5x Margin, Both Directions, and Time Filter
Author: Your Name
Date: 2025-08-16
"""

import yfinance as yf
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time

# Define trading hours (IST)
TRADE_START = time(9, 30)
TRADE_END = time(15, 0)

# Brokerage + slippage functions
def groww_cost(value, intraday=True):
    pct = value * 0.001  # 0.1%
    brk = max(5, min(20, pct))
    stt = value * 0.00025 if intraday else value * 0.001
    stamp = value * 0.00003 if intraday else value * 0.00015
    exch = value * 0.0000297
    sebi = value * 0.000001
    gst = 0.18 * (brk + exch + sebi)
    return brk + stt + stamp + exch + sebi + gst

def slip(price, vol, buy=True):
    s = max(0.10, min(2.0, price * 0.0002 * max(0.5, 2e5 / max(vol, 1))))
    return price + s if buy else price - s

# Main trading class with modifications
class Scalper:
    def __init__(self, cash=1_000_00, risk=0.01, day_loss_limit=0.01):
        self.cash0 = cash
        self.cash = cash
        self.risk = risk
        self.day_loss_limit = day_loss_limit
        self.trades = []
        self.current_position = None
        self.day_start_time = None

        # Tracking daily PnL
        self.day_pnl = 0
        self.loss_count = 0

    def is_trading_time(self, timestamp):
        local_time = timestamp.time()
        return TRADE_START <= local_time <= TRADE_END

    def ind(self, df):
        c = df['Close'].values
        out = {}
        out['rsi'] = talib.RSI(c, 14)
        out['macd'], out['macds'], _ = talib.MACD(c, 12, 26, 9)
        out['bbU'], out['bbM'], out['bbL'] = talib.BBANDS(c, 20, 2, 2)
        out['ema9'] = talib.EMA(c, 9)
        out['ema21'] = talib.EMA(c, 21)
        return out

    def generate_signal(self, index, df, ind):
        rsi = ind['rsi'][index]
        macd = ind['macd'][index]
        macs = ind['macds'][index]
        close = df['Close'].iloc[index]
        ema9 = ind['ema9'][index]
        ema21 = ind['ema21'][index]

        # Entry signals
        long_signal = False
        short_signal = False

        # Example condition: RSI crossing thresholds and MACD confirmation
        if rsi < 30 and macd > macs and close > ema9:
            long_signal = True
        elif rsi > 70 and macd < macs and close < ema9:
            short_signal = True

        return 'long' if long_signal else 'short' if short_signal else None

    def execute_trade(self, df, index, signal):
        timestamp = df.index[index]
        price = df['Close'].iloc[index]
        volume = df['Volume'].iloc[index]

        # Only trade within allowed hours
        if not self.is_trading_time(timestamp):
            return

        # Check daily loss limits
        if abs(self.day_pnl) >= self.cash0 * self.day_loss_limit:
            return  # Stop trading today

        # Determine number of shares with 5x margin
        max_sl = self.cash * self.risk * 5
        qty = int(max_sl / (price * 0.015))
        qty = max(qty, 1)  # at least 1

        # Entry logic
        if self.current_position is None and signal:
            # Open position
            entry_price = slip(price, volume, buy=True)
            cost = groww_cost(entry_price * qty)
            if cost <= self.cash:
                self.cash -= cost
                self.current_position = {
                    'entry_price': entry_price,
                    'qty': qty,
                    'direction': signal,
                    'entry_time': timestamp
                }
                print(f"Entered {signal} at {entry_price:.2f} qty {qty} old cash {self.cash:.2f}")
        # Exit logic
        elif self.current_position and self.current_position['direction'] != signal:
            exit_price = slip(price, volume, buy=(signal=='long'))
            entry_price = self.current_position['entry_price']
            qty = self.current_position['qty']
            gross_pnl = (exit_price - entry_price) * qty
            net_cost = groww_cost(exit_price * qty)
            total_pnl = gross_pnl - net_cost + groww_cost(entry_price * qty)
            self.cash += net_cost + total_pnl  # Add PnL to cash
            # Log trade
            self.trades.append({
                'date': timestamp.date(),
                'entry_time': self.current_position['entry_time'],
                'exit_time': timestamp,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'qty': qty,
                'direction': self.current_position['direction'],
                'pnl': total_pnl,
                'entry': 'long' if self.current_position['direction']=='long' else 'short',
                'exit': 'long' if signal=='long' else 'short'
            })
            # Reset position
            self.current_position = None
            print(f"Exited at {exit_price:.2f} for pnl {total_pnl:.2f}, cash {self.cash:.2f}")

    def backtest(self, symbol, date_str):
        # Download data for the trading day
        start_dt = datetime.strptime(date_str, "%Y-%m-%d")
        end_dt = start_dt + timedelta(days=1)
        # df = yf.download(symbol, start=start_dt, end=end_dt, interval='5m', progress=False)
        df = yf.download(symbol, period="60d", interval='5m', progress=False, multi_level_index=False, auto_adjust=True)

        if df.empty:
            print(f"No data for {symbol} on {date_str}")
            return

        # Filter to trading hours
        df['Time'] = df.index.time
        df = df[df['Time'].between(TRADE_START, TRADE_END)]

        # Indicator computation
        ind = self.ind(df)

        for i in range(max(0, df.index.get_loc(pd.Timestamp(start_dt))), len(df)):
            timestamp = df.index[i]
            if not self.is_trading_time(timestamp):
                continue
            signal = self.generate_signal(i, df, ind)
            self.execute_trade(df, i, signal)

        # Close any open position at end of day
        if self.current_position:
            last_idx = len(df) - 1
            self.execute_trade(df, last_idx, None)

        # Summary
        print(f"\nTotal PnL for {symbol} on {date_str}: ₹{self.cash - self.cash0:.2f}")

    def save_trades(self, filename):
        # Convert list of trades to DataFrame
        df_trades = pd.DataFrame(self.trades)
        df_trades.to_csv(filename, index=False)

# Usage example:
if __name__ == "__main__":
    date_str = "2025-08-16"  # For same day
    symbols = ['RELIANCE.NS']  # Add your NSE stocks
    trader = Scalper()
    for sym in symbols:
        trader.backtest(sym, date_str)
    trader.save_trades('nse_scalp_results.csv')


KeyError: Timestamp('2025-08-16 00:00:00')